**Test for Python Developer – API Data Extraction role**

1. Uses google search to get insta profile ids

In [ ]:

import requests
from bs4 import BeautifulSoup
import os
import time
import re
import pandas as pd  
import urllib
from PIL import Image

**step 1 :** Get insta user links from google search

In [ ]:
PROJECT_DIR = '/content/drive/MyDrive/DataExtraction'
DATA_DIR = os.path.join(PROJECT_DIR,'ScrappedData')
INSTA_DATA_DIR = os.path.join(DATA_DIR,'InstaData')

In [ ]:
if not os.path.exists(PROJECT_DIR):
  os.mkdir(PROJECT_DIR)

if not os.path.exists(DATA_DIR):
  os.mkdir(DATA_DIR)

if not os.path.exists(INSTA_DATA_DIR):
  os.mkdir(INSTA_DATA_DIR)

In [ ]:
search_url = 'https://www.google.com/search?q=site%3Ainstagram.com+and+intitle%3A%40'

In [ ]:
# function to get insta profile urls from bs4 element
def get_profile_url(heading_element):
  link = heading_element.find_parent('a')
  link_href = link.get('href')
  profile_link = link_href.split('=')[1]
  profile_link = profile_link.split('/&')[0]
  return profile_link


In [ ]:
insta_user_ids = []
insta_user_urls = []

search_items = 10
#loop over search pages
limit_pages = 100
page_no = 1

 #loop for 100 google search pages
for page in range(limit_pages):
  #get the page
  data = requests.get(search_url)
  print("Scrapping data from page : {}".format(page_no))
  #create bs4 soup object
  soup = BeautifulSoup(data.content, "html.parser")
  #extract headings
  headings = soup.find_all( 'h3' )
  #loop over headings of curr page 
  for heading in headings:
    if '@' in heading.getText():
      profile_url = get_profile_url(heading)
      insta_id = re.findall('\(@.*?\)', heading.getText())
      if len(insta_id) > 0:
        insta_id = insta_id[0].replace('(','').replace(')','')
        insta_user_ids.append(insta_id)
        insta_user_urls.append(profile_url)
  
  #update search url for new page
  search_url = search_url + '&start='+str(search_items)
  search_items = search_items + 10
  page_no = page_no + 1
  print("Insta URLS saved : {}".format(len(insta_user_ids)))
  time.sleep(3.6)

#save as csv file

dict = {'insta_id': insta_user_ids, 'profile_urls': insta_user_urls}  

df = pd.DataFrame(dict) 
    
# saving the dataframe 
df.to_csv(os.path.join(DATA_DIR,'insta_users_data1.csv')) 




Scrapping data from page : 1
Insta URLS saved : 7
Scrapping data from page : 2
Insta URLS saved : 16
Scrapping data from page : 3
Insta URLS saved : 22
Scrapping data from page : 4
Insta URLS saved : 29
Scrapping data from page : 5
Insta URLS saved : 38
Scrapping data from page : 6
Insta URLS saved : 47
Scrapping data from page : 7
Insta URLS saved : 56
Scrapping data from page : 8
Insta URLS saved : 64
Scrapping data from page : 9
Insta URLS saved : 72
Scrapping data from page : 10
Insta URLS saved : 81
Scrapping data from page : 11
Insta URLS saved : 86
Scrapping data from page : 12
Insta URLS saved : 94
Scrapping data from page : 13
Insta URLS saved : 104
Scrapping data from page : 14
Insta URLS saved : 113
Scrapping data from page : 15
Insta URLS saved : 122
Scrapping data from page : 16
Insta URLS saved : 130
Scrapping data from page : 17
Insta URLS saved : 137
Scrapping data from page : 18
Insta URLS saved : 144
Scrapping data from page : 19
Insta URLS saved : 153
Scrapping data 

**Step 2:** use obtained insta profile urls and get all required data using instascrape

In [ ]:
#load the csv file

insta_users_info = pd.read_csv(os.path.join(DATA_DIR,'insta_users_data1.csv'))

insta_users_info.head()

,Unnamed: 0,insta_id,profile_urls
0,0,@theoldmanandthethree,https://www.instagram.com/theoldmanandthethree
1,1,@mtv,https://www.instagram.com/mtv
2,2,@valkyrae,https://www.instagram.com/valkyrae
3,3,@lilyjcollins,https://www.instagram.com/lilyjcollins
4,4,@hm,https://www.instagram.com/hm


In [ ]:
insta_ids = insta_users_info['insta_id']

In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,428 kB]
Ge

In [ ]:
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": "sessionid=33726898013%3AiXbdXjOUcT2dHd%3A27;"
}

In [ ]:
!pip install insta-scrape


In [ ]:
from instascrape import *


In [ ]:
def get_image(url,name):
   urllib.request.urlretrieve(url, name)
   i = Image.open(name)
   i.thumbnail((100, 100), Image.LANCZOS)
   i.save(name)




In [ ]:
def get_twitter_acc(username):
  search_query = 'https://www.google.com/search?q=site%3Atwitter.com+and+intitle:'+username
  data = requests.get(search_query)
  soup = BeautifulSoup(data.content, "html.parser")
  #extract headings
  headings = soup.find_all( 'h3' )
  #loop over headings of curr page 
  twitter_link = ' '
  twitter_id= ' '
  for heading in headings:
    if username in heading.getText().lower():
      twitter_id = re.findall('\(@.*?\)', heading.getText())
      if len(twitter_id) > 0:
        twitter_id= twitter_id[0].replace('(','').replace(')','')
        twitter_link = get_profile_url(heading).split('&')[0]
        return twitter_link



In [ ]:
def get_youtube_acc(username):
  search_query = 'https://www.google.com/search?q='+username
  data = requests.get(search_query)
  soup = BeautifulSoup(data.content, "html.parser")
  #extract headings
  headings = soup.find_all( 'h3' )
  #loop over headings of curr page 
  youtube_link = ' '
  for heading in headings:
    parent_el = str(heading.parent)
    if 'youtube.com' in parent_el and 'watch' not in parent_el and username in parent_el:
      youtube_link = parent_el.split('=')[2].split('&')[0]
      return youtube_link

In [ ]:
#xls
!pip3 install xlsxwriter

     |████████████████████████████████| 148 kB 5.3 MB/s 


In [ ]:
import xlsxwriter

In [ ]:
for insta_id in insta_ids:
  insta_id = insta_id.replace('@','')
  print("Scrapping details from id :"+str(insta_id))
  user = Profile(insta_id)
  user.scrape(headers=headers)
  follower_count = user.followers
  following_count = user.following
  username = user.username
  full_name = user.full_name
  url = user.url
  verfified = str(user.is_verified)
  profile_pic_url = user.profile_pic_url
  category = user.category_enum
  num_posts = user.posts
  biography = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['biography']
  connected_fb_page = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['connected_fb_page']
  web_page = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['external_url']
  is_business_account = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['is_business_account']
  twitter_acc_link = get_twitter_acc(full_name) 
  youtube_acc = get_youtube_acc(full_name)
  if (is_business_account):
    acc_type = 'Business'
  else:
    acc_type = ' '
  get_image(profile_pic_url,'profile_pic.png')

  workbook = xlsxwriter.Workbook(str(INSTA_DATA_DIR)+'/insta_'+str(insta_id)+'.xlsx')
  worksheet = workbook.add_worksheet()
  worksheet.write('A1', 'Name')
  worksheet.write('B1', 'UserName')
  worksheet.write('C1', 'Age')
  worksheet.write('D1', 'Topics')
  worksheet.write('E1', 'Followers')
  worksheet.write('F1', 'Following')
  worksheet.write('G1', 'AVG Likes')
  worksheet.write('H1', 'Avg. Comments')
  worksheet.write('I1', 'Eng Rate')
  worksheet.write('J1', 'Total Posts')
  worksheet.write('K1', 'Email')
  worksheet.write('L1', 'Website')
  worksheet.write('M1', 'Bio')
  worksheet.write('N1', 'Gender')
  worksheet.write('O1', 'Location')
  worksheet.write('P1', 'Language')
  worksheet.write('Q1', 'Youtube')
  worksheet.write('R1', 'Twitter')
  worksheet.write('S1', 'TikTok')
  worksheet.write('T1', 'Facebook')
  worksheet.write('U1', 'Profile Picture url')
  worksheet.write('V1', 'Profile Picture')
  worksheet.write('W1', 'Verified')
  worksheet.write('X1', 'Account type')

  worksheet.set_row(1,50)
  worksheet.set_column_pixels(22,2,60)

  worksheet.write('A2', full_name)
  worksheet.write('B2', username)
  worksheet.write('C2', 'NA')
  worksheet.write('D2', 'NA')
  worksheet.write('E2', follower_count)
  worksheet.write('F2', following_count)
  worksheet.write('G2', ' ')
  worksheet.write('H2', ' ')
  worksheet.write('I2', ' ')
  worksheet.write('J2', num_posts)
  worksheet.write('K2', ' ')
  worksheet.write('L2', web_page)
  worksheet.write('M2', biography)
  worksheet.write('N2', ' ')
  worksheet.write('O2', ' ')
  worksheet.write('P2', ' ')
  worksheet.write('Q2', youtube_acc)
  worksheet.write('R2', twitter_acc_link)
  worksheet.write('S2', ' ')
  worksheet.write('T2', connected_fb_page)
  worksheet.write('U2', profile_pic_url)
  worksheet.insert_image('V2', 'profile_pic.png')
  worksheet.write('W2', verfified)
  worksheet.write('X2',acc_type )

  worksheet.set_column('A:A', 30)

  #save recent postsheading

  worksheet.write('A11', 'Posts')
  worksheet.write('B11', 'Description')
  worksheet.write('C11', 'Hashtags')
  worksheet.write('D11', 'Tags')
  worksheet.write('E11', 'Likes')
  worksheet.write('F11', 'Comments')
  worksheet.write('G11', 'Image/Video URL')
  worksheet.write('H11', 'Post URL')
  recent_posts = user.get_recent_posts()
  row = 12
  for post in recent_posts:
    post_url = post.url
    post_media_url = post.display_url
    get_image(post_media_url,str(row)+'.png')
    try:
      post_description = post.json_dict['edge_media_to_caption']['edges'][0]['node']['text']
    except:
      post_description = ' '
    hashtags = regex = re.findall("#(\w+)", post_description)
    hashtags = "#"+",#".join(hashtags)
    num_likes = post.likes
    num_comments = post.comments

    worksheet.set_row(row-1,80)
    worksheet.set_column_pixels(1,row,100)


    worksheet.insert_image('A'+str(row), str(row)+'.png')
    worksheet.write('B'+str(row), post_description)
    worksheet.write('C'+str(row), hashtags)
    worksheet.write('D'+str(row), ' ')
    worksheet.write('E'+str(row), num_likes)
    worksheet.write('F'+str(row), num_comments)
    worksheet.write('G'+str(row), post_media_url)
    worksheet.write('H'+str(row), post_url)
    row = row +1

  workbook.close()





Scrapping details from id :theoldmanandthethree


/usr/local/lib/python3.7/dist-packages/instascrape/core/_static_scraper.py:136: MissingCookiesWarning: Request header does not contain cookies! It's recommended you pass at least a valid sessionid otherwise Instagram will likely redirect you to their login page.
  MissingCookiesWarning


Scrapping details from id :mtv
Scrapping details from id :valkyrae
Scrapping details from id :lilyjcollins
Scrapping details from id :hm
Scrapping details from id :kyliejenner
Scrapping details from id :kendalljenner
Scrapping details from id :kateupton
Scrapping details from id :christojeanneclaude
Scrapping details from id :lizgillz
Scrapping details from id :justlikethatmax
Scrapping details from id :rafaelloandco
Scrapping details from id :channingtatum
Scrapping details from id :haileesteinfeld
Scrapping details from id :t22felton
Scrapping details from id :keke
Scrapping details from id :fatjoe
Scrapping details from id :dazed
Scrapping details from id :asabopp
Scrapping details from id :haileybieber
Scrapping details from id :kellyclarkson
Scrapping details from id :danandshay
Scrapping details from id :nikitadragun
Scrapping details from id :hoskelsa
Scrapping details from id :rickandmorty
Scrapping details from id :oliviawilde
Scrapping details from id :lilireinhart
Scrapping 